# "The Eye Of The Typer" Dataset Rerun

Visualize using `rerun`.

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from typing import cast
from pathlib import Path
from pprint import pprint
from datetime import timedelta

import polars as pl
import rerun as rr

from eott_dataset import *
from eott_dataset.rerun import *

In [ ]:
ds = EyeTyperDataset()

for source, schema in ds.describe(Source.MOUSE).items():
    print_schema(source, schema)

In [ ]:
participant = ds.participant(1)
print(f"Participant {participant.pid:>02d}")

In [ ]:
record = 3
recording_id = f"{participant.pid:02d}-{record:02d}"

rr.init("EOTT", recording_id=recording_id, spawn=True)
rerun_log_participant(participant, record)

entries_iter = participant.iter_timeline(record=record)
for entry in entries_iter:
    rr.set_time_seconds("offset", entry["offset"].total_seconds())

    match entry["source"]:
        case "tobii":
            rerun_log_tobii(cast(TobiiEntry, entry["data"]), screen=participant.screen_res)

        case "mouse":
            rerun_log_mouse(cast(MouseEntry, entry["data"]))

        case "screen" | "webcam":
            entity = rr.Image(cast(FrameEntry, entry["data"])["frame"])
            if entry["source"] == "screen":
                entity = entity.compress()
            rr.log(entry["source"], entity)

    del entry